In [27]:
import pandas as pd


In [28]:
# Load data
df = pd.read_csv("xpaths_20260122_165932.csv")
df = df[["Label", "Action", "Value"]]
df


,Label,Action,Value
0,firstName,click,NaN
1,firstName,change,PRAKYATH
2,email,click,NaN
3,email,change,PRAK@GMAIL.COM
4,middleName,click,NaN
5,middleName,change,M
6,mobileNumber,click,NaN
7,mobileNumber,change,123-4567
8,lastName,click,NaN
9,lastName,change,CHAN


In [29]:
# edge case dictionary (value + description pairs)

edge_cases = {
    'name': [
        ('', 'empty'),
        ('A'*20, 'too long' ),
        ('@#$%^!&*()_=+/*-', 'special characters'),
        ('1234567890', 'numbers only'),    
    ],

    'email': [
        ('', 'empty'),
        ('invalid', 'no @ symbol'),
        ('test@', 'incomplete'),
        ('@gmail.com', 'no username'),
    ],

    'phone': [
        ('', 'empty'),
        ('ancdef', 'letters'),
        ('1', 'too short'),
    ],
    'text': [
        ('', 'empty'),
        ('@#$%^%^&*', 'special characters'),
    ],
}

In [30]:
FIELD_PATTERNS = {
    'name': ['name', 'firstname', 'lastname', 'middlename'],
    'email': ['email', 'mail'],
    'phone': ['phone', 'mobile', 'telephone', 'mobilenumber'],
}

def detect_field_type(label):
    label_lower = label.lower().replace('_', '').replace('-', '')
    for field_type, patterns in FIELD_PATTERNS.items():
        for pattern in patterns:
            if pattern in label_lower:
                return field_type
    return 'text'


In [31]:
print(detect_field_type("firstName"))
print(detect_field_type("email"))
print(detect_field_type("mobilenumber"))

name
email
phone


In [36]:
def generate_test_scenarios(df):
    # clean data
    df_clean = df.copy()
    df_clean['Value'] = df_clean.apply(
        lambda row: 'Click' if row['Action'] == 'click' else row['Value'],
        axis = 1
    )

    elements = df_clean['Label'].tolist()
    actions = df_clean['Action'].tolist()
    original_values = df_clean['Value'].tolist()

    #step2: Find change indices
    change_indices = [i for i, action in enumerate(actions) if action == 'change']

    #ste3: Create Scenarios lsit
    scenarios = []

    #Row 0: Perfect sample
    scenarios.append({
        'description': 'Perfect template (all valid)',
        'values': original_values.copy()
    })

    #step4: For each input fields include edge case
    for idx in change_indices:
        label = elements[idx]
        field_type = detect_field_type(label)

        #Get edge cases for this field
        cases = edge_cases.get(field_type, edge_cases['text'])

        for test_value, test_desc in cases:
            #copy original value
            new_values = original_values.copy()
            #change only ONE field
            new_values[idx] = test_value
            #create description
            description = f"{label} - {test_desc}"

            scenarios.append({
                'description': description,
                'values': new_values
            })

    #step5: Build final DataFrame
    rows = []
    rows.append(["Description"] + elements) # header row
    rows.append(['Action'] + actions)

    for scenario in scenarios:
        rows.append([scenario['description']] + scenario['values'])

    result_df = pd.DataFrame(rows)
    return result_df


In [37]:
result = generate_test_scenarios(df)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Description,firstName,firstName,email,email,middleName,middleName,mobileNumber,mobileNumber,lastName,lastName,Select Branch,100 - TECU - MARABELLA BRANCH,Add profile picture,Capture,Cancel,svg,Save & Continue
1,Action,click,change,click,change,click,change,click,change,click,change,click,click,click,click,click,click,click
2,Perfect template (all valid),Click,PRAKYATH,Click,PRAK@GMAIL.COM,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
3,firstName - empty,Click,,Click,PRAK@GMAIL.COM,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
4,firstName - too long,Click,AAAAAAAAAAAAAAAAAAAA,Click,PRAK@GMAIL.COM,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
5,firstName - special characters,Click,@#$%^!&*()_=+/*-,Click,PRAK@GMAIL.COM,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
6,firstName - numbers only,Click,1234567890,Click,PRAK@GMAIL.COM,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
7,email - empty,Click,PRAKYATH,Click,,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
8,email - no @ symbol,Click,PRAKYATH,Click,invalid,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click
9,email - incomplete,Click,PRAKYATH,Click,test@,Click,M,Click,123-4567,Click,CHAN,Click,Click,Click,Click,Click,Click,Click


In [39]:
import pandas as pd
import numpy as np

In [41]:
# Read as text first to see the issue
with open("test_scenarios (4).csv", "r") as f:
    print(f.read())


Description,firstName,firstName,email,email,middleName,mobileNumber,mobileNumber,lastName,lastName,Select Branch,300 - TECU - PT. FORTIN BRANCH,Save & Continue
Perfect template,Click,PRAKYATH,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
firstName - empty,Click,,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
firstName - special,Click,@#$%^&,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
firstName - numbers,Click,12345,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
firstName - too long,Click,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
email - empty,Click,PRAKYATH,Click,,M,Click,666-6666,Click,CHAN,Click,Click,Click
email - special,Click,PRAKYATH,Click,@#$%^&,M,Click,666-6666,Click,CHAN,Click,Click,Click
email - numbers,Click,PRAKYATH,Click,12345,M,Click,666-6666,Click,CHAN,Click,Click,Click
email - too long,Click,PRAKYATH,Click,AAAAAAAAAAAAAAAAAAAAAA

In [45]:
data = pd.read_csv("test_scenarios (5).csv", on_bad_lines='skip')

In [48]:
# data1 = pd.read_csv("test_scenarios (3).csv")
# data1

In [46]:
data

,Description,firstName,firstName.1,email,email.1,middleName,mobileNumber,mobileNumber.1,lastName,lastName.1,Select Branch,300 - TECU - PT. FORTIN BRANCH,Save & Continue
0,Perfect template,Click,PRAKYATH,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
1,firstName - empty,Click,NaN,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
2,firstName - special,Click,@#$%^&,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
3,firstName - numeric,Click,12345,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
4,firstName - long,Click,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,Click,P@GMAIL.COM,M,Click,666-6666,Click,CHAN,Click,Click,Click
5,email - empty,Click,PRAKYATH,Click,NaN,M,Click,666-6666,Click,CHAN,Click,Click,Click
6,email - special,Click,PRAKYATH,Click,@#$%^&,M,Click,666-6666,Click,CHAN,Click,Click,Click
7,email - numeric,Click,PRAKYATH,Click,12345,M,Click,666-6666,Click,CHAN,Click,Click,Click
8,email - long,Click,PRAKYATH,Click,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,M,Click,666-6666,Click,CHAN,Click,Click,Click
9,middleName - empty,Click,PRAKYATH,Click,P@GMAIL.COM,NaN,Click,666-6666,Click,CHAN,Click,Click,Click


In [24]:
print(data)

               Description firstName  \
0         Perfect template     Click   
1        firstName - empty     Click   
2      firstName - special     Click   
3      firstName - numbers     Click   
4         firstName - long     Click   
5            email - empty     Click   
6          email - special     Click   
7          email - numbers     Click   
8             email - long     Click   
9       middleName - empty     Click   
10    middleName - special     Click   
11    middleName - numbers     Click   
12       middleName - long     Click   
13    mobileNumber - empty     Click   
14  mobileNumber - special     Click   
15  mobileNumber - numbers     Click   
16     mobileNumber - long     Click   
17        lastName - empty     Click   
18      lastName - special     Click   
19      lastName - numbers     Click   
20         lastName - long     Click   

                                          firstName.1  email  \
0                                            PRAKYATH  

In [55]:
updated_data = pd.read_csv("test_scenarios (9).csv")

In [56]:
updated_data

,Description,firstName,firstName.1,email,email.1,middleName,middleName.1,mobileNumber,mobileNumber.1,lastName,lastName.1,Select Branch,300 - TECU - PT. FORTIN BRANCH,+1868,TRINIDAD AND TOBAGO (+1868),Add profile picture,Capture,Capture Photo,Save & Continue
0,Perfect template,Click,PRAKYATH,Click,A@GMAIL.COM,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
1,firstName - empty,Click,NaN,Click,A@GMAIL.COM,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
2,firstName - special,Click,@#$%^&,Click,A@GMAIL.COM,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
3,firstName - numeric,Click,12345,Click,A@GMAIL.COM,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
4,firstName - long,Click,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,Click,A@GMAIL.COM,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
5,email - empty,Click,PRAKYATH,Click,NaN,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
6,email - special,Click,PRAKYATH,Click,@#$%^&,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
7,email - numeric,Click,PRAKYATH,Click,12345,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
8,email - long,Click,PRAKYATH,Click,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,Click,M,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
9,middleName - empty,Click,PRAKYATH,Click,A@GMAIL.COM,Click,NaN,Click,777-7777,Click,CHAN,Click,Click,Click,Click,Click,Click,Click,Click
